## Linear Regression

## Ols

In [59]:
import numpy as np
x = np.array([1,3,5,8,10])
y = np.array([10,20,30,35,45])

### Eq -1

In [60]:
#wo = 0
#w1 = 0
#y = wo + w1*x

In [61]:
x_y_bar = (x*y).mean()
x_bar = x.mean()
y_bar = y.mean()
x_sqr_bar = (x*x).mean()

In [62]:
w1 = ((x_y_bar) - x_bar*y_bar) / (x_sqr_bar - x_bar*x_bar)
w1

3.646616541353384

In [63]:
wo = y_bar - (w1*x_bar)
wo

8.308270676691723

In [64]:
y_hat = wo+(w1*x)

In [65]:
y_hat

array([11.95488722, 19.2481203 , 26.54135338, 37.48120301, 44.77443609])

## Eq - 2

In [66]:
w1 = ((x - x_bar) * (y-y_bar)).sum() / ((x - x_bar) * (x - x_bar)).sum()
w1

3.6466165413533833

In [67]:
wo = y_bar - w1*x_bar
wo

8.30827067669173

### 1. Normal Equation Method

θ=(XTX)^−1 XT y

In [70]:
def predict_sales(x, weight, bias):
    return weight*x + bias


In [88]:
import numpy as np
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)


In [89]:
X_b = np.c_[np.ones((100, 1)), X]
X_b.shape

(100, 2)

In [90]:

theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
theta_best

array([[4.04981596],
       [2.90628349]])

### 2. Gradient descent

### Cost function

𝑀𝑆𝐸 =1 / 𝑁 (∑︁ 𝑛 𝑖=1 (𝑦𝑖 − (𝑚𝑥𝑖 + 𝑏))^2)

    𝑁 is the total number of observations (data points)

* 1 /  𝑁 ∑︀𝑛 𝑖=1 is the mean

*  𝑦𝑖 is the actual value of an observation and 𝑚𝑥𝑖 + 𝑏 is our prediction

In [6]:
def cost_function(x, y, weight, bias):
    companies = len(x)
    total_error = 0.0
    for i in range(companies):
        total_error += (y[i] - (weight*x[i] + bias))**2
    return total_error / companies

In [7]:
def update_weights(radio, sales, weight, bias, learning_rate):
    weight_deriv = 0
    bias_deriv = 0
    companies = len(radio)
    for i in range(companies):
        # Calculate partial derivatives
        # -2x(y - (mx + b))
        weight_deriv += -2*radio[i] * (sales[i] - (weight*radio[i] + bias))
        # -2(y - (mx + b))
        bias_deriv += -2*(sales[i] - (weight*radio[i] + bias))
        # We subtract because the derivatives point in direction of steepest ascent
        weight -= (weight_deriv / companies) * learning_rate
        bias -= (bias_deriv / companies) * learning_rate
    return weight, bias